# Etapa 1: Importação das bibliotecas

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [10]:
!pip install bert-for-tf2
!pip install sentencepiece

In [11]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__
# !pip install tensorflow==2.2.0-rc3

'2.3.0'

# Etapa 2: Pré-processamento

## Carregamento dos arquivos

In [12]:
cols = ["sentiment", "id", "date", "query", "user", "text"]

data = pd.DataFrame(columns = cols)
for i in range(0, 10):
  df = pd.read_csv(f'https://raw.githubusercontent.com/alexgf/bert-classificacao-sentimentos/master/db/database_{i}.csv',
                   header = None,
                   names = cols,
                   engine='python',
                   encoding='latin1')
  data = pd.concat([data, df])

In [13]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [14]:
data.shape

(1600000, 2)

In [15]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Pré-processamento

### Limpeza

In [16]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [17]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [18]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [19]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [20]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [21]:
encode_sentence("My dog likes strawberries.")

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [22]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [23]:
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

In [24]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [25]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [26]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [27]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)

In [28]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [29]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [30]:
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [31]:
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [32]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [33]:
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0) 
           ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935541e-01, -4.10335153e-01, -9.65755045e-01,
          9.07317877e-01,  8.12913775e-01, -1.74174339e-01,
          9.11234558e-01,  3.41952085e-01, -8.74521315e-01,
         -9.99989271e-01, -7.78409958e-01,  9.69385207e-01,
          9.86160457e-01,  6.36963248e-01,  9.48631227e-01,
         -7.51193166e-01, -4.58339661e-01, -7.08104372e-01,
          4.62098330e-01, -6.57927096e-01,  7.60414422e-01,
          9.99994814e-01, -3.96861106e-01,  3.44166011e-01,
          6.16488576e-01,  9.94400024e-01, -7.76633739e-01,
          9.38316464e-01,  9.59452212e-01,  7.32879102e-01,
         -6.93436623e-01,  2.93080389e-01, -9.93785501e-01,
         -1.64551780e-01, -9.67019558e-01, -9.95549500e-01,
          5.32935381e-01, -6.88060820e-01,  1.34715075e-02,
          2.98195817e-02, -9.18356419e-01,  4.20526296e-01,
          9.99988914e-01,  2.52676070e-01,  6.06235385e-01,
         -3.50750059e-01, -1.00000000e+00,  4.975

In [34]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
              sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [35]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [36]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [37]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Etapa 3: Construção do modelo

In [38]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Etapa 4: Treinamento

In [39]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [40]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [41]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
checkpoint_path = "/content/drive/My Drive/BERT_udemy/embedding"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [45]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [46]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
  40623/Unknown - 4259s 105ms/step - loss: 0.3571 - accuracy: 0.8431Checkpoint saved at /content/drive/My Drive/BERT_udemy/embedding.
40623/40623 [==============================] - 4262s 105ms/step - loss: 0.3571 - accuracy: 0.8431
Epoch 2/5
40623/40623 [==============================] - 4239s 104ms/step - loss: 0.3496 - accuracy: 0.8468
Epoch 3/5
40623/40623 [==============================] - 4252s 105ms/step - loss: 0.3435 - accuracy: 0.8495
Epoch 4/5
40623/40623 [==============================] - 4243s 104ms/step - loss: 0.3374 - accuracy: 0.8519
Epoch 5/5
40623/40623 [==============================] - 4237s 104ms/step - loss: 0.3320 - accuracy: 0.8540


# Etapa 5: Avaliação do modelo

In [47]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 256s 57ms/step - loss: 0.3543 - accuracy: 0.8523
[0.35434025526046753, 0.8523224592208862]


In [48]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))


In [49]:
get_prediction("This actor is very bad.")

Output of the model: [[0.01026896]]
Predicted sentiment: negative


In [55]:
get_prediction("Great your parents are here")

Output of the model: [[0.8608974]]
Predicted sentiment: positive


In [52]:
get_prediction("I like nothing.")

Output of the model: [[0.19937466]]
Predicted sentiment: negative
